# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5: Extended Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [20]:
import sympy as sp
import numpy as np

In [21]:
#@title Estado de sistema
X, Y, Z, Xp, Yp, Zp = sp.symbols('X Y Z \dot{X} \dot{Y} \dot{Z}')
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [Xp],
    [Yp],
    [Zp],
])

sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [22]:
#@title Matriz A "transitionMatrix"
dt = sp.Symbol('\Delta t')
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0],
    [0, 1, 0, 0, dt, 0],
    [0, 0, 1, 0, 0, dt],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol('\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]]))

In [23]:
#@title Ecuacion de tranformacion de estado
# el estado en k, es el resultado de multiplicar A por el estado anterior
sp.Eq(XX, sp.MatMul(AA, XX, evaluate=False), evaluate=False)

Eq(Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]), Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]])*Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [24]:
#@title "Z" Medicion
x, y, r, xp, yp = sp.symbols('x y \dot{x} \dot{y} r')
# La medicion esta compuesta por las coordenadas de la camara y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [x],
    [y],
    [xp],
    [yp],
    [r],
])
sp.Eq(sp.Symbol('\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[      x],
[      y],
[\dot{y}],
[      r],
[\dot{x}]]))

In [25]:
# @title Funcion h
R_real = sp.symbols('Rr')
h_x_ = sp.Matrix([
    [X / Z],
    [Y / Z],
    [(Xp + x * Zp) / Z],
    [(Yp + y * Zp) / Z],
    [R_real / Z],
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X}')), h_x_, evaluate=False)

Eq(h(\mathbb{X}), Matrix([
[                    X/Z],
[                    Y/Z],
[(\dot{X} + \dot{Z}*x)/Z],
[(\dot{Y} + \dot{Z}*y)/Z],
[                   Rr/Z]]))

In [26]:
# @title Funcion h
h_x__ = sp.Matrix([
    [X / Z],
    [Y / Z],
    [(Xp + (X / Z) * Zp) / Z],
    [(Yp + (Y / Z) * Zp) / Z],
    [R_real / Z],
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X}')), h_x__, evaluate=False)

Eq(h(\mathbb{X}), Matrix([
[                      X/Z],
[                      Y/Z],
[(X*\dot{Z}/Z + \dot{X})/Z],
[(Y*\dot{Z}/Z + \dot{Y})/Z],
[                     Rr/Z]]))

In [27]:
# @title Funcion h
h_x = sp.Matrix([
    [X / Z],
    [Y / Z],
    [(Z * Xp + X * Zp) / Z ** 2],
    [(Z * Yp + Y * Zp) / Z ** 2],
    [R_real / Z],
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X}')), h_x, evaluate=False)

Eq(h(\mathbb{X}), Matrix([
[                         X/Z],
[                         Y/Z],
[(X*\dot{Z} + Z*\dot{X})/Z**2],
[(Y*\dot{Z} + Z*\dot{Y})/Z**2],
[                        Rr/Z]]))

In [28]:
# @title Matriz H es el Jacobian de h respecto al estado
HH = h_x.jacobian(XX)
sp.Eq(sp.Symbol('\mathbb{H}x'), HH, evaluate=False)

Eq(\mathbb{H}x, Matrix([
[         1/Z,            0,                                       -X/Z**2,   0,   0,      0],
[           0,          1/Z,                                       -Y/Z**2,   0,   0,      0],
[\dot{Z}/Z**2,            0, \dot{X}/Z**2 - 2*(X*\dot{Z} + Z*\dot{X})/Z**3, 1/Z,   0, X/Z**2],
[           0, \dot{Z}/Z**2, \dot{Y}/Z**2 - 2*(Y*\dot{Z} + Z*\dot{Y})/Z**3,   0, 1/Z, Y/Z**2],
[           0,            0,                                      -Rr/Z**2,   0,   0,      0]]))

In [29]:
HH * XX

Matrix([
[                                                                               0],
[                                                                               0],
[2*X*\dot{Z}/Z**2 + Z*(\dot{X}/Z**2 - 2*(X*\dot{Z} + Z*\dot{X})/Z**3) + \dot{X}/Z],
[2*Y*\dot{Z}/Z**2 + Z*(\dot{Y}/Z**2 - 2*(Y*\dot{Z} + Z*\dot{Y})/Z**3) + \dot{Y}/Z],
[                                                                           -Rr/Z]])

In [30]:
#@title Matriz de calibracion.
K_cam = sp.Matrix([
    [1.3778036814997304e+03, 0, 4.0002681782947193e+02],
    [0, 1.3778036814997304e+03, 3.00096061319675721e+02],
    [0, 0, 1],
])
sp.Eq(sp.Symbol('K'), K_cam, evaluate=False)

Eq(K, Matrix([
[1377.80368149973,                0, 400.026817829472],
[               0, 1377.80368149973, 300.096061319676],
[               0,                0,                1]]))

In [31]:
#@title Matriz de calibracion (inversa)
K_cam_inv = K_cam.inv()
sp.Eq(sp.Symbol('K^{-1}'), K_cam_inv, evaluate=False)

Eq(K^{-1}, Matrix([
[0.000725792805918116,                    0, -0.290336586554948],
[                   0, 0.000725792805918116, -0.217807562390183],
[                   0,                    0,                  1]]))

In [32]:
# imprimir la innversa de K en formato de numpy
print(np.array(K_cam_inv).astype(np.float64))

[[ 7.25792806e-04  0.00000000e+00 -2.90336587e-01]
 [ 0.00000000e+00  7.25792806e-04 -2.17807562e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [33]:
OO = K_cam_inv * sp.Matrix([
    [399],
    [299],
    [1],
])
OO

Matrix([
[-0.000745256993619159],
[ -0.00079551342066575],
[                    1]])

In [34]:
K_cam * OO

Matrix([
[399.0],
[299.0],
[    1]])

In [35]:
f = K_cam[0, 0]
(Z * Rp) / f

NameError: name 'Rp' is not defined

In [ ]:
# test cholesky
A = sp.Matrix([
    [1, 0.5, 0.5],
    [0.5, 1, 0.5],
    [0.5, 0.5, 1],
])
# cholesky
L = A.cholesky()
L * L.T